# Deliverable 3

Trying to get more than 75% of accuracy

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [4]:
application_df = application_df.drop(['EIN', 'NAME'], 1)

/tmp/ipykernel_271065/88848161.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['EIN', 'NAME'], 1)


In [5]:
atc = application_df.APPLICATION_TYPE.value_counts()

In [6]:
# Determine which values to replace if counts are less than ...?
replace_app = atc[atc < 500].index

# Replace in dataframe
for app in replace_app:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, "Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
cv = application_df.CLASSIFICATION.value_counts()

In [8]:
# Determine which values to replace if counts are less than ..?
rc = list(cv[cv < 500].index)

# Replace in dataframe
for cls_ in rc:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls_, "Other")

In [9]:
app_category = application_df.dtypes[application_df.dtypes == "object"].index

In [13]:
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encodedf = pd.DataFrame(enc.fit_transform(application_df[app_category]))

# Add the encoded variable names to the dataframe
encodedf.columns = enc.get_feature_names(app_category)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encodedf, left_index=True, right_index=True)
application_df = application_df.drop(columns = app_category)
# Split our preprocessed data into our features and target arrays
t1 = application_df["IS_SUCCESSFUL"].values
t2 = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(t2, t1, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/tmp/ipykernel_271065/2383433740.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  t2 = application_df.drop(["IS_SUCCESSFUL"],1).values


In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_l1 =  80
hidden_nodes_l2 = 30
hidden_nodes_l3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_l1, 
                          input_dim=number_input_features, 
                          activation="relu")
    )

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_l2, 
                          activation="sigmoid"))

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, 
                          activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3600      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


2021-09-19 23:44:51.104078: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-19 23:44:51.106439: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "weights.{epoch:02d}.hdf5"

# Compile the model
nn.compile(loss="binary_crossentropy", 
           optimizer="adam", 
           metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,
                   y_train,epochs=30,
                   callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2021-09-19 23:46:55.213197: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-19 23:46:55.231118: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2096020000 Hz


Epoch 1/30
804/804 [==============================] - 3s 3ms/step - loss: 0.5729 - accuracy: 0.7207
Epoch 2/30
196/804 [======>.......................] - ETA: 1s - loss: 0.5558 - accuracy: 0.7313
Epoch 00002: saving model to weights.02.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5552 - accuracy: 0.7320
Epoch 3/30
389/804 [=============>................] - ETA: 1s - loss: 0.5585 - accuracy: 0.7245
Epoch 00003: saving model to weights.03.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5528 - accuracy: 0.7307
Epoch 4/30
578/804 [====================>.........] - ETA: 0s - loss: 0.5507 - accuracy: 0.7331
Epoch 00004: saving model to weights.04.hdf5
804/804 [==============================] - 3s 3ms/step - loss: 0.5507 - accuracy: 0.7323
Epoch 5/30
757/804 [===========================>..] - ETA: 0s - loss: 0.5492 - accuracy: 0.7313
Epoch 00005: saving model to weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5492